In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Ex01").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/03/01 18:31:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#EXERCICIO SUGERIDO
'''
    Através das bases no diret dados realizar as consltas:

    -> Consulta na ordem de colunas: Nome, Estado e Status
    -> Consulta que mostra apenas os clientes do status Platinum e gold
    -> Demonstre quanto cada status de cliente representa em vendas

'''

In [2]:
#import data

clientes = spark.read.parquet("dados/Clientes.parquet")
itensVendas = spark.read.parquet("dados/ItensVendas.parquet")
produtos = spark.read.parquet("dados/Produtos.parquet")
vendas = spark.read.parquet("dados/Vendas.parquet")
vendedores = spark.read.parquet("dados/Vendedores.parquet")

clientes.show()

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
|        6|       Adérito Bahía|    MA|     M|  Silver|
|        7|       Aida Dorneles|    RN|     F|  Silver|
|        8|   Alarico Quinterno|    AC|     M|  Silver|
|        9|    Alberto Cezimbra|    AM|     M|  Silver|
|       10|    Alberto Monsanto|    RN|     M|    Gold|
|       11|       Albino Canela|    AC|     M|  Silver|
|       12|     Alceste Varanda|    RR|     F|  Silver|
|       13|  Alcides Carvalhais|    RO|     M|  Silver|
|       14|        Aldo Martins|    GO|     M|  Silver|
|       15|   Alexandra Tabares|    MG|     F|  

In [7]:
# -> Consulta na ordem de colunas: Nome, Estado e Status

clientes.select("Cliente", "Estado", "Status").show()


+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [15]:
# -> Consulta que mostra apenas os clientes do status Platinum e gold
from pyspark.sql import functions as Func

clientes.select("Cliente", "Estado", "Status").where((Func.col("Status") == "Silver") | (Func.col("Status") == "Platinum")).show()



+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
|           Ana Homem|    RN|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [20]:
# -> Demonstre quanto cada status de cliente representa em vendas

from pyspark.sql import functions as Func
from pyspark.sql.types import *

#relacionamento das tabelas vendas e clientes
vendas_clientes = vendas.join(clientes, vendas.ClienteID == clientes.ClienteID, "inner")

# Mostra quantidade de vendas por status
vendas_clientes.groupBy("Status").count().orderBy("count", ascending = False).show()

# Mostra soma de receita por status

vendas_clientes.groupBy("Status").sum("Total").orderBy("sum(Total)", ascending = False).show()


+--------+-----+
|  Status|count|
+--------+-----+
|  Silver|  395|
|    Gold|    3|
|Platinum|    2|
+--------+-----+

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+

